<a href="https://colab.research.google.com/github/Sunaina50/NutriShield-Allergen-Detection-System/blob/main/Synapse_2_0_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
# Sample training data for symptoms and corresponding allergies
training_symptoms = [
    "hives, swelling", "shortness of breath", "stomach pain",
    "itching", "nausea", "rash", "vomiting", "headache"
]
training_labels = [
    "Nut or Seafood Allergy", "Severe Allergy (Consult a Doctor)",
    "Dairy or Gluten Allergy", "Egg or Soy Allergy",
    "Seafood or Dairy Allergy", "Gluten or Nut Allergy",
    "Food Poisoning or Severe Allergy", "Preservative or Additive Sensitivity"
]

In [3]:
# Train ML model
vectorizer = TfidfVectorizer()
model = make_pipeline(vectorizer, MultinomialNB())
model.fit(training_symptoms, training_labels)

# Save model
joblib.dump(model, "allergy_predictor.pkl")

# Load trained model
def predict_allergy_ml(symptoms):
    model = joblib.load("allergy_predictor.pkl")
    prediction = model.predict([symptoms])[0]
    return prediction


In [4]:
# Allergen detection
def detect_allergens(food_name):
    allergen_db = {
        "peanut butter": "🔴 High Risk: Contains Peanuts",
        "milkshake": "🔴 High Risk: Contains Milk",
        "bread": "🟠 Moderate Risk: Contains Gluten",
        "tofu": "🟠 Moderate Risk: Contains Soy",
        "shrimp": "🔴 High Risk: Contains Seafood",
        "omelette": "🟠 Moderate Risk: Contains Eggs",
        "walnuts": "🔴 High Risk: Contains Tree Nuts",
        "pasta": "🟢 Low Risk: Contains Carbs, Fiber, and Protein",
        "lobster": "🔴 High Risk: Contains Shellfish",
        "hummus": "🟠 Moderate Risk: Contains Sesame"
    }
    return allergen_db.get(food_name.lower(), "✅ No allergens detected.") if food_name else "N/A"



In [5]:
# Suggest dietary alternatives
def suggest_alternatives(food_name):
    alternatives = {
        "peanut butter": "🌻 Sunflower seed butter, Pumpkin seed butter",
        "milkshake": "🥛 Almond milk, Soy milk, Oat milk",
        "bread": "🍞 Rice flour bread, Corn tortillas, Quinoa bread",
        "tofu": "🌰 Coconut aminos, Pea protein, Chickpea flour",
        "shrimp": "🍗 Chicken, Tofu, Mushrooms",
        "omelette": "🥚 Chia seeds, Flax seeds, Banana puree",
        "walnuts": "🌻 Pumpkin seeds, Sunflower seeds, Hemp seeds",
        "pasta": "🍝 Rice noodles, Quinoa pasta, Lentil pasta",
        "lobster": "🍄 Mushrooms, Chicken, Plant-based seafood alternatives",
        "hummus": "🥄 Chia seeds, Flaxseeds, Sunflower seeds"
    }
    return alternatives.get(food_name.lower(), "❌ No suitable alternatives found.") if food_name else "N/A"


In [6]:
# Generate diet report
def generate_diet_report(symptoms, food_name=None):
    if not symptoms:
        return "⚠ Please provide symptoms to proceed."

    detected = detect_allergens(food_name) if food_name else "N/A"
    alternatives = suggest_alternatives(food_name) if food_name else "N/A"
    predicted_allergy = predict_allergy_ml(symptoms)

    report = (
        "📋 **Personalized Diet Report**\n"
        "| **Category**             | **Details** |\n"
        "|:------------------------|:-------------|\n"
        f"| **Predicted Allergy**     | {predicted_allergy} |\n"
        f"| **Identified Allergens**  | {detected} |\n"
        f"| **Safe Alternatives**     | {alternatives} |\n"
        "| **Suggested Meal Plan**   | Fresh vegetables, fruits, lean proteins, whole grains |\n"
    )

    return report



In [7]:
# Web Interface
def web_interface(choice, symptoms, food_name=None):
    if not symptoms:
        return "⚠ Symptoms are required for accurate prediction!"

    if choice == "1️⃣ Check allergens in a food item":
        return detect_allergens(food_name) if food_name else "⚠ Please provide a food item."
    elif choice == "2️⃣ Predict allergy from symptoms using ML":
        return predict_allergy_ml(symptoms)
    elif choice == "3️⃣ Generate personalized diet report":
        return generate_diet_report(symptoms, food_name)
    else:
        return "⚠ Invalid choice. Please select a valid option."

In [8]:
# Gradio App Design with Enhanced UI
custom_css = """
    body { background-color: #FFFFFF; color: #000000; font-family: 'Arial', sans-serif; }  /* White Background & Black Text */
    h1 { color: #4CAF50; text-align: center; }
    h3 { color: #4CAF50; text-align: center; }  /* Centered Heading */
    p { text-align: center; color: #000000; }   /* Black Text for Paragraph */
    .gradio-button { background-color: #4CAF50; color: #FFFFFF; border: none; }
"""

with gr.Blocks(css=custom_css) as app:
    gr.HTML(
        """
        <h1>🩺 <b>NutriShield : Allergy Detection System 🌿</b></h1>
        <div style="text-align: center;">
            <h3>🔎 Detect allergens, suggest alternatives, and predict allergies! 🚀</h3>
        </div>
        """,
    )

    choice_input = gr.Dropdown(
        ["1️⃣ Check allergens in a food item",
         "2️⃣ Predict allergy from symptoms using ML",
         "3️⃣ Generate personalized diet report"],
        label="🔹 Choose an Option"
    )

    with gr.Row():
        symptoms_input = gr.Textbox(label="🤒 Enter Symptoms")
        food_input = gr.Textbox(label="🍽️ Enter Food Name (Optional)")

    result_output = gr.Markdown()
    submit_button = gr.Button("🩺 Generate Report", variant="primary")

    submit_button.click(web_interface,
                        inputs=[choice_input, symptoms_input, food_input],
                        outputs=result_output)

    gr.Markdown(
        """
        <hr>
        <p style="text-align: center; font-weight: bold; color: #4CAF50;">
        ❤️ Developed for your health and safety! 🚑
        </p>
        """
    )

app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1a6df8c086c12e5a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
